# FIT5148 - Distributed Databases and Big Data
# Assignment 2 Task B- Solution Workbook

**Instructions:**

    You will be using Python 3.
    You can introduce new cells as necessary.

**Your details**

    Name: Moon Byeong Kim

    Student ID: 26389126

    Name: Hun Tae Lee
    Student ID: 22568735

Let's get started

## Import Libraries

In [1]:
import pymongo

## Task B1. Justify Data Model Design 

There are two ways of modelling relationship connecting two datasets, Embedding and Referencing. In this assignment, we used the Embedded document structure, embedding fire data sets into climate data sets.
Reason:
1. This climate and fire data are not too big. If there are milions of data, referencing would be good.
    - small subdocuments. If subdocument are large, it would be impossible to display the data efficiently; in this case, reference would be better.
    - less than 16 MB
2. Performance of embedded model is faster than referencing model, and also easy to implement.
3. Possible more support from library
4. Data does not change regularly.
    - Referecing is suitable when subdocument need to be changed regular bases since it just use id as reference.
5. Easy to read in single glance
6. We embed fire into climate since climate has unique date attribute, while fire has multiple documents that has same date. so used date as join attribute.

## Task B2 & B3
### B2. Create New Database in MongoDB
### B3. Python Program that Load Data to New Database Created

In [2]:
from pymongo import MongoClient
from pprint import pprint

def readFromFile(fname):
    f = open(fname)
    header = list()
    data = list()
    # get header of csv files from f
    temp = f.readline().strip().split(',')
    header = [x.strip(' ') for x in temp]
    # get data of csv files from f
    for line in f:
        #get rid of /n and spaces
        temp = line.strip().split(',')
        temp = [x.strip(' ') for x in temp]
        data.append(temp)
    print('FileName: ', fname, ', Load: completed.')
    return header,data

if __name__ == "__main__":
    
    cHeader, climateData = readFromFile("ClimateData-Part1.csv")
    fHeader, fireData= readFromFile("FireData-Part1.csv") 
    result = list()
    for c in climateData:
        fires = list()
        for f in fireData:
            tempf = dict()
            # find fireData which has same date as climateData for embedding
            if c[1] == f[6]:
                for i in range(0,8):
                    # confidence and surface temperature(c) are integer
                    if (i == 5) or (i == 7):
                        tempf.update({fHeader[i]:int(f[i])})
                    # Latitude, longitude, Surface temperature(k), and power are float
                    elif (i == 0) or (i == 1) or (i == 2) or (i == 4):
                        tempf.update({fHeader[i]:float(f[i])})
                    else:
                        tempf.update({fHeader[i]:f[i]})
                fires.append(tempf)
        tempc = dict()
        for i in range(0,9):
            #Air temperature(c) is integer
            if (i == 2):
                tempc.update({cHeader[i]:int(c[i])})
            #Relative Humidity, windspeed, max wind speed are float
            elif (i == 3) or (i == 4) or (i == 5):
                tempc.update({cHeader[i]:float(c[i])})
            else:
                tempc.update({cHeader[i]:c[i]})
        tempc.update({"fire":fires})
        result.append(tempc)
        
    client = MongoClient()
   
    #TaskB q2: create database name as "taskB_db"
    db = client.taskB_db
    #set collection name as dmc
    dmc = db.dmc.drop()
    dmc = db.dmc
    result = dmc.insert_many(result)
    print(result, " : Data Model successfully inserted into new Database, taskB_db, as collection name, dmc.")
                            

FileName:  ClimateData-Part1.csv , Load: completed.
FileName:  FireData-Part1.csv , Load: completed.
<pymongo.results.InsertManyResult object at 0x7f51e7984688>  : Data Model successfully inserted into new Database, taskB_db, as collection name, dmc.


## Task B4

** a2. Find climate data on 15th December 2017 **

In [3]:
#find climate data on 15th of December 2017
results = dmc.find({"Date":"2017-12-15"})
for result in results:
    pprint(result)

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'MAX': '74.5*',
 'MIN': '53.1',
 'Max Wind Speed': 14.0,
 'Precipitation': '0.00I',
 'Relative Humidity': 52.0,
 'Station': '948702',
 'WindSpeed  (knots)': 7.1,
 '_id': ObjectId('5b010c4aa9141f29c0c0d78b'),
 'fire': [{'Confidence': 92,
           'Date': '2017-12-15',
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.627,
           'Longitude': 149.33,
           'Power': 23.1,
           'Surface Temperature (Celcius)': 42,
           'Surface Temperature (kelvin)': 315.9},
          {'Confidence': 78,
           'Date': '2017-12-15',
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.658,
           'Longitude': 149.339,
           'Power': 17.3,
           'Surface Temperature (Celcius)': 36,
           'Surface Temperature (kelvin)': 309.8},
          {'Confidence': 51,
           'Date': '2017-12-15',
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.623,


** a3. Find the latitude, longitude and confidence when the surface temperature (Celcius) was 65 (Celicus) and 100 (Celcius)       **

In [4]:
#Find the latitude, longitude and confidence when the surface temperature(C) was between 65(C) and 100(C)
results = dmc.find({"fire.Surface Temperature (Celcius)":{"$gte":65,"$lte":100}},{"fire.Latitude":1,"fire.Longitude":1,"fire.Confidence":1})

for result in results:
    pprint(result)


{'_id': ObjectId('5b010c4aa9141f29c0c0d673'),
 'fire': [{'Confidence': 100, 'Latitude': -37.223, 'Longitude': 147.9431},
          {'Confidence': 100, 'Latitude': -37.2252, 'Longitude': 147.9363},
          {'Confidence': 94, 'Latitude': -37.2284, 'Longitude': 147.9187},
          {'Confidence': 97, 'Latitude': -37.6572, 'Longitude': 142.0703},
          {'Confidence': 81, 'Latitude': -37.6592, 'Longitude': 142.058},
          {'Confidence': 55, 'Latitude': -36.2544, 'Longitude': 148.0353},
          {'Confidence': 54, 'Latitude': -37.2197, 'Longitude': 147.9621},
          {'Confidence': 74, 'Latitude': -37.2128, 'Longitude': 147.9308}]}
{'_id': ObjectId('5b010c4aa9141f29c0c0d675'),
 'fire': [{'Confidence': 94, 'Latitude': -37.4157, 'Longitude': 147.0084},
          {'Confidence': 84, 'Latitude': -37.0193, 'Longitude': 148.1459},
          {'Confidence': 100, 'Latitude': -37.4229, 'Longitude': 147.027},
          {'Confidence': 80, 'Latitude': -37.0055, 'Longitude': 148.1582},
       

** a4. Find surface temperature(Celcius), air temperature(Celcius), relative humidity and maximum wind speed on 15th and 16th of Decmber 2017 **

In [5]:
#Find surface temperature(C), air temperature(C), relative humidity and 
#maximum wind speed on 15th and 16th of Decmber 2017
results = dmc.find({"$or":[{"Date":"2017-12-15"},{"Date":"2017-12-16"}]},{"Date":1,"fire.Surface Temperature (Celcius)":1,"Air Temperature(Celcius)":1,"Relative Humidity":1,"Max Wind Speed":1})
for result in results:
    pprint(result)

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14.0,
 'Relative Humidity': 52.0,
 '_id': ObjectId('5b010c4aa9141f29c0c0d78b'),
 'fire': [{'Surface Temperature (Celcius)': 42},
          {'Surface Temperature (Celcius)': 36},
          {'Surface Temperature (Celcius)': 38},
          {'Surface Temperature (Celcius)': 40}]}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-16',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 '_id': ObjectId('5b010c4aa9141f29c0c0d78c'),
 'fire': [{'Surface Temperature (Celcius)': 43},
          {'Surface Temperature (Celcius)': 33},
          {'Surface Temperature (Celcius)': 54},
          {'Surface Temperature (Celcius)': 73},
          {'Surface Temperature (Celcius)': 55},
          {'Surface Temperature (Celcius)': 75},
          {'Surface Temperature (Celcius)': 55},
          {'Surface Temperature (Celcius)': 66},
          {'Surface Temperature (Celcius)': 56},
          {'Surface Temperature (Celcius)': 60},
   

** a5. Find datetime, air temperature(Celcius), surface temperature(Celcius) and confidence when the confidence is between 80 and 100 **

In [6]:
#Find datetime, air temperature(C), surface temperature(C) and confidence when the confidence is between 80 and 100
results = dmc.find({"fire.Confidence":{"$gte":80,"$lte":100}},{"fire.Datetime":1,"Air Temperature(Celcius)":1,"fire.Surface Temperature (Celcius)":1,"fire.Confidence":1})
for result in results:
    pprint(result)

{'Air Temperature(Celcius)': 20,
 '_id': ObjectId('5b010c4aa9141f29c0c0d66f'),
 'fire': [{'Confidence': 87,
           'Datetime': '2017-03-06T05:06:30',
           'Surface Temperature (Celcius)': 62},
          {'Confidence': 85,
           'Datetime': '2017-03-06T05:06:20',
           'Surface Temperature (Celcius)': 59}]}
{'Air Temperature(Celcius)': 19,
 '_id': ObjectId('5b010c4aa9141f29c0c0d670'),
 'fire': [{'Confidence': 88,
           'Datetime': '2017-03-07T04:16:10',
           'Surface Temperature (Celcius)': 64}]}
{'Air Temperature(Celcius)': 23,
 '_id': ObjectId('5b010c4aa9141f29c0c0d672'),
 'fire': [{'Confidence': 86,
           'Datetime': '2017-03-09T13:23:40',
           'Surface Temperature (Celcius)': 41},
          {'Confidence': 54,
           'Datetime': '2017-03-09T03:57:00',
           'Surface Temperature (Celcius)': 44},
          {'Confidence': 78,
           'Datetime': '2017-03-09T03:56:50',
           'Surface Temperature (Celcius)': 55}]}
{'Air Temperatur

** a6. Find top 10 records with highest surface temperature(Celcius) **

In [7]:
#Find top 10 records with highest surface temperature(C)
#results = dmc.find().sort("fire.Surface Temperature (Celcius)",-1).limit(10)
results = dmc.aggregate([{"$project":{"Date":1,"fire.Surface Temperature (Celcius)":1}},{"$unwind":"$fire"},{"$sort":{"fire.Surface Temperature (Celcius)":-1}},{"$limit":10}])
for result in results:
    pprint(result)

{'Date': '2017-04-18',
 '_id': ObjectId('5b010c4aa9141f29c0c0d69a'),
 'fire': {'Surface Temperature (Celcius)': 124}}
{'Date': '2017-04-04',
 '_id': ObjectId('5b010c4aa9141f29c0c0d68c'),
 'fire': {'Surface Temperature (Celcius)': 123}}
{'Date': '2017-05-01',
 '_id': ObjectId('5b010c4aa9141f29c0c0d6a7'),
 'fire': {'Surface Temperature (Celcius)': 122}}
{'Date': '2017-03-18',
 '_id': ObjectId('5b010c4aa9141f29c0c0d67b'),
 'fire': {'Surface Temperature (Celcius)': 121}}
{'Date': '2017-04-04',
 '_id': ObjectId('5b010c4aa9141f29c0c0d68c'),
 'fire': {'Surface Temperature (Celcius)': 120}}
{'Date': '2017-04-13',
 '_id': ObjectId('5b010c4aa9141f29c0c0d695'),
 'fire': {'Surface Temperature (Celcius)': 120}}
{'Date': '2017-05-13',
 '_id': ObjectId('5b010c4aa9141f29c0c0d6b3'),
 'fire': {'Surface Temperature (Celcius)': 120}}
{'Date': '2017-04-18',
 '_id': ObjectId('5b010c4aa9141f29c0c0d69a'),
 'fire': {'Surface Temperature (Celcius)': 120}}
{'Date': '2017-04-04',
 '_id': ObjectId('5b010c4aa9141f2

** a7. Find the number of fire in each day. You are required to only display total number of fire and the date in the output **

In [8]:
#Find the number of fire in each day. You are required to only display total number of fire and the date in the output
results = dmc.aggregate([{"$group":{"_id":"$Date", "Count":{"$sum":{"$size":"$fire"}}}}])
for result in results:
    pprint(result)

{'Count': 0, '_id': '2018-01-01'}
{'Count': 0, '_id': '2017-12-30'}
{'Count': 0, '_id': '2017-12-28'}
{'Count': 1, '_id': '2017-12-24'}
{'Count': 0, '_id': '2017-12-19'}
{'Count': 1, '_id': '2017-12-14'}
{'Count': 1, '_id': '2017-12-13'}
{'Count': 1, '_id': '2017-12-12'}
{'Count': 2, '_id': '2017-12-10'}
{'Count': 4, '_id': '2017-12-09'}
{'Count': 0, '_id': '2017-12-04'}
{'Count': 0, '_id': '2017-12-02'}
{'Count': 8, '_id': '2017-11-29'}
{'Count': 0, '_id': '2017-11-24'}
{'Count': 0, '_id': '2017-12-07'}
{'Count': 2, '_id': '2017-11-22'}
{'Count': 0, '_id': '2017-11-16'}
{'Count': 5, '_id': '2017-11-13'}
{'Count': 4, '_id': '2017-11-11'}
{'Count': 10, '_id': '2017-11-09'}
{'Count': 0, '_id': '2017-11-04'}
{'Count': 0, '_id': '2017-11-03'}
{'Count': 0, '_id': '2017-11-02'}
{'Count': 0, '_id': '2017-11-01'}
{'Count': 0, '_id': '2017-10-29'}
{'Count': 5, '_id': '2017-10-27'}
{'Count': 0, '_id': '2017-10-25'}
{'Count': 0, '_id': '2017-10-24'}
{'Count': 1, '_id': '2017-10-23'}
{'Count': 0, 

** a8. Find the average surface temperature(C) for each day. You are required to only display average surface temperature(C) and the date in the output **

In [9]:
#q5_a8
#Find the average surface temperature(C) for each day. You are required to only display average surface temperature(C)
#and the date in the output
results = dmc.aggregate([{"$unwind":"$fire"},{"$group":{"_id":"$Date","avg":{"$avg":"$fire.Surface Temperature (Celcius)"}}}])
for result in results:
    pprint(result)

{'_id': '2017-12-24', 'avg': 32.0}
{'_id': '2017-12-21', 'avg': 46.0}
{'_id': '2017-12-15', 'avg': 39.0}
{'_id': '2017-12-13', 'avg': 60.0}
{'_id': '2017-12-10', 'avg': 46.0}
{'_id': '2017-12-09', 'avg': 58.25}
{'_id': '2017-11-30', 'avg': 52.41935483870968}
{'_id': '2017-11-28', 'avg': 42.0}
{'_id': '2017-11-22', 'avg': 61.5}
{'_id': '2017-11-21', 'avg': 59.0}
{'_id': '2017-11-14', 'avg': 52.0}
{'_id': '2017-11-11', 'avg': 46.25}
{'_id': '2017-11-09', 'avg': 61.3}
{'_id': '2017-11-08', 'avg': 45.5}
{'_id': '2017-11-05', 'avg': 58.5}
{'_id': '2017-10-28', 'avg': 56.0}
{'_id': '2017-10-23', 'avg': 38.0}
{'_id': '2017-10-20', 'avg': 50.0}
{'_id': '2017-10-18', 'avg': 52.166666666666664}
{'_id': '2017-10-17', 'avg': 51.6}
{'_id': '2017-10-16', 'avg': 36.0}
{'_id': '2017-10-15', 'avg': 72.66666666666667}
{'_id': '2017-10-08', 'avg': 41.0}
{'_id': '2017-10-06', 'avg': 44.0}
{'_id': '2017-10-02', 'avg': 43.57142857142857}
{'_id': '2017-10-01', 'avg': 48.25}
{'_id': '2017-09-29', 'avg': 43.0}

In [38]:
# Using Pool library for multi-processcing 
from multiprocessing import Pool

#parallel processing
data = dmc.find()
dataList = []
for doc in data:
    dataList.append(doc)


# Round-robin data partitioning function
# Need round robin partitioning
def rr_partition(data, n):
    """
    Perform data partitioning on data
    
    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors
    
    Return:
    result -- the partitioned subset of D
    """
    
    result = []
    for i in range(n):
        result.append([])
    
    #Calculate the number of elements to be allocated to each bin
    n_bin = len(data)/n
    
    #For each bin, perform the following:
    for index, element in enumerate(data):
        #Calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)

        result[index_bin].append(element)
    
    return result


# Binary search function
# Need binary search
def binary_search(data, key):
    """
    Perform binary search on data for the given key
    
    Arguments:
    data -- an input dataset which is a list
    key -- an query record
    
    Return:
    result -- the position of searched record
    """
    
    matched_record = None
    position = -1 #not found position
    
    lower = 0
    middle = 0
    upper = len(data)-1
    
    while (lower <= upper): 
        #calculate middle: the half of lower and upper
        middle = int((lower + upper)/2)
        #find dictionary value which has key named 'Date'
        if(data[middle].get('Date') > key): 
            upper = middle-1
        elif(data[middle].get('Date') < key): 
            lower = middle+1
        elif(data[middle].get('Date') == key): 
            matched_record = data[middle]
            position = middle
            break
            
    return position, matched_record

# Parallel searching algorithm for exact match
def parallel_search_exact(data, query, n_processor, m_partition, m_search):
    """
    Perform parallel search for exact match on data for the given key
    partitioning is round-robin.
    Arguments:
    data -- an input dataset which is a list
    query -- a query record
    n_processor -- the number of parallel processor
    m_partition -- a data partitining method
    m_search -- a search method
    
    Return:
    results -- the matched record information
    """
    
    results = []
    
    # pool: a pyhon method enabling parallel processing.
    # We need to set the number of processes to n_processsor
    # Which means that the Pool class will allow 'n_processor' processes
    # running at the same time
    pool = Pool(processes=n_processor)
    
    # For round-robin partitioning method
    # Perform data partitioning first
    DD = m_partition(data, n_processor)
    for d in DD: # Perform parallel search on all data prartitions
        result = pool.apply(m_search,[d,query])
        
        # will only append the result found, not not-found
        if(result[0]!=-1):
            results.append(result) 
    
    return results

# Output of using the parallel_search_exact function, round-robin partition, binary_search
ans = parallel_search_exact(dataList, '2017-12-15',4,rr_partition, binary_search)
pprint(ans)

[(87,
  {'Air Temperature(Celcius)': 18,
   'Date': '2017-12-15',
   'MAX': '74.5*',
   'MIN': '53.1',
   'Max Wind Speed': 14.0,
   'Precipitation': '0.00I',
   'Relative Humidity': 52.0,
   'Station': '948702',
   'WindSpeed  (knots)': 7.1,
   '_id': ObjectId('5b010c4aa9141f29c0c0d78b'),
   'fire': [{'Confidence': 92,
             'Date': '2017-12-15',
             'Datetime': '2017-12-15T13:17:17',
             'Latitude': -37.627,
             'Longitude': 149.33,
             'Power': 23.1,
             'Surface Temperature (Celcius)': 42,
             'Surface Temperature (kelvin)': 315.9},
            {'Confidence': 78,
             'Date': '2017-12-15',
             'Datetime': '2017-12-15T13:17:17',
             'Latitude': -37.658,
             'Longitude': 149.339,
             'Power': 17.3,
             'Surface Temperature (Celcius)': 36,
             'Surface Temperature (kelvin)': 309.8},
            {'Confidence': 51,
             'Date': '2017-12-15',
             'Da

In [11]:
#Drop database taskB_db
client = MongoClient()
client.drop_database('taskB_db')